Explore TwitterAPI
====

Based on this code: http://geduldig.github.io/TwitterAPI/faulttolerance.html

In [61]:
reset -fs

In [55]:
from itertools import islice
from time import sleep

from credentials import credentials
from ftfy import fix_text
from textblob import TextBlob
from TwitterAPI import TwitterAPI

In [56]:
creds = credentials.require(['access_token', 
                             'access_token_secret', 
                             'consumer_key',
                             'consumer_secret'])

api = TwitterAPI(creds.consumer_key,
                 creds.consumer_secret,
                 creds.access_token,
                 creds.access_token_secret)

In [57]:
def tweets(query):
    "A generator of tweets"
    while True:
        try:
            iterator = api.request('search/tweets', {'q':query}).get_iterator()
            for item in iterator:
                if 'text' in item:
                    yield(item['text'])
                elif 'disconnect' in item:
                    event = item['disconnect']
                    if event['code'] in [2,5,6,7]:
                        # something needs to be fixed before re-connecting
                        raise Exception(event['reason'])
                    else:
                        # temporary interruption, re-try request
                        break
        except TwitterRequestError as e:
            if e.status_code < 500:
                # something needs to be fixed before re-connecting
                raise
            else:
                # temporary interruption, re-try request
                pass
        except TwitterConnectionError:
            # temporary interruption, re-try request
            pass

In [58]:
# Grab some tweets

tweets_about_beer = tweets(query='beer')
list(islice(tweets_about_beer, 5))

['@janefletcherer  fing spirt. Atl. AND they made me chug my beer before I got in line and then they deboarded us! More pissed about that.',
 'Drinking a Pacifico Clara by @Grupo_Modelo - https://t.co/oTpr3Z2Aur',
 "I guess a couple games of beer pong then that's it 😴",
 'Drinking a Jack Savage by @bigwoodbrewery at @barleyandvinemn — https://t.co/OKo2c2LZ8o #photo',
 'RT @MattLinzbach: @10centpower good beer out this way.  Check out @LupineBrewing in Delano also.']

In [59]:
# Stream tweets and tag the words as they go by
tweets_about_pizza = tweets(query='pizza')

for _ in range(2):
    blob = TextBlob(fix_text(next(tweets_about_beer)))
    print("Tweet: \t", blob.raw)
    print("POS Tags:",) # Part of Speech 
    [print("\t", token) for token in blob.tags]
    print('#'*40)
    sleep(1)

Tweet: 	 RT @373KInc: It's Payday!  Before you buy that beer, consider a small donation to Lost & Found Youth.  https://t.co/uAyfsYbBSC https://t.co…
POS Tags:
	 ('RT', 'NNP')
	 ('@', 'VBD')
	 ('373KInc', 'CD')
	 ('It', 'PRP')
	 ("'s", 'VBZ')
	 ('Payday', 'JJ')
	 ('Before', 'IN')
	 ('you', 'PRP')
	 ('buy', 'VBP')
	 ('that', 'IN')
	 ('beer', 'NN')
	 ('consider', 'VB')
	 ('a', 'DT')
	 ('small', 'JJ')
	 ('donation', 'NN')
	 ('to', 'TO')
	 ('Lost', 'NNP')
	 ('&', 'CC')
	 ('Found', 'NNP')
	 ('Youth', 'NNP')
	 ('https', 'NN')
	 ('//t.co/uAyfsYbBSC', 'JJ')
	 ('https', 'NN')
	 ('//t.co…', 'NN')
########################################
Tweet: 	 Drinking a Negra Modelo by @Grupo_Modelo - https://t.co/UEZIAEZVv1
POS Tags:
	 ('Drinking', 'VBG')
	 ('a', 'DT')
	 ('Negra', 'NNP')
	 ('Modelo', 'NNP')
	 ('by', 'IN')
	 ('@', 'NNP')
	 ('Grupo_Modelo', 'NNP')
	 ('https', 'NN')
	 ('//t.co/UEZIAEZVv1', 'NN')
########################################


---
__The internet also has Rainbow Butterfly Unicorn Kittens:__

![](https://s-media-cache-ak0.pinimg.com/736x/bf/3f/4c/bf3f4c4e4cbc909f957f939bb6bc7cc6.jpg)